In [ ]:
#from polynomial import *
%run polynomial.py
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import math

Using polynomials and shifts

The following function defines the list
\\[f(n) = [∞, 1!, 2!,\ldots, (n-1)!]\\]

In [ ]:
def texp(k):
    return np.hstack([np.inf, np.cumprod(np.arange(k-1)+1)])

Now define an approximation of \\(\exp(x)-1 = \sum_{k=1}^{∞}x^k/k!\\) by
\\[
p_n(x) := \sum_{k=1}^{n-1} \frac{x^k}{k!}
\\]

In [ ]:
def poly_expm1(k):
    return Polynomial(1./texp(k))

Compare the approximation \\(p_{20}\\) with the true function on the imaginary axis.

In [ ]:
def plot_both_imaginary(k=20, x=.9*3*np.pi):
    y = 1
    #plt.figure(figsize=(zoom*2*x,zoom*y))
    xs = np.linspace(-x,x,200)
    zs = 1j*xs
    expected = np.expm1(zs)
    computed = poly_expm1(k)(zs)
    plt.subplot(1,2,1)
    for vals in [computed, expected]:
        plt.plot(np.real(vals), np.imag(vals))
        plt.axis('scaled')
    plt.subplot(1,2,2)
    plt.plot(xs, np.imag(expected))
    plt.plot(xs, np.imag(computed))
    plt.axis('scaled')
    plt.grid(ls='-', alpha=.2)

In [ ]:
plot_both_imaginary()

In [ ]:
plot_both_imaginary(10, 4)

In [ ]:
for s,k in enumerate(range(15,40,5)):
    p = poly_expm1(k)
    zeros = p.zeros()
    plt.plot(np.real(zeros), np.imag(zeros), ls='', marker='o', label="{}".format(k), markersize=s+3)
plt.legend(bbox_to_anchor=(1.2,1.))
plt.grid(ls='-', alpha=.2)
plt.axis('scaled')

In [ ]:
def plot_abs(k):
    p = poly_expm1(k)
    zeros = p.zeros()
    #def get_grid(x0,x1,y0,y1)
    margin = 1
    xmin = np.min(np.real(zeros)) - margin
    xmax = np.max(np.real(zeros)) + margin
    ymin = np.min(np.imag(zeros)) - margin
    ymax = np.max(np.imag(zeros)) + margin
    res = 100
    xs,ys = np.meshgrid(np.linspace(xmin,xmax,res), np.linspace(ymin,ymax,res))
    zs = 10*np.log10(np.abs(p(xs + 1j*ys))+np.finfo(float).eps)
    
    CS=plt.contour(xs,ys,zs)
    plt.clabel(CS, inline=1, fontsize=5)
    plt.plot(np.real(zeros), np.imag(zeros), ls='', marker='o')
    plt.axis('scaled')
    plt.grid(ls='-', alpha=.2)
    plt.title(k)


In [ ]:
plot_abs(32)

In [ ]:
plot_abs(10)

In [ ]:
plot_abs(44)

Using a Taylor basis

The basis is $ω^k X^k/k!$. A vector with coordinates $x_i$ represents the polynomial
\\[
x_0 + x_1 ωX + x_2 ω^2X^2/2! + \cdots = ∑_i x_i \frac{ω^i X^i}{i!}
\\]
The shift operation is thus
\begin{align}
X e_i = (i+1)/ω e_{i+1}
\end{align}

We look for the roots of the polynomial of degree $n$
\begin{align}
P = ∑_i^{n} p_i e_i
\end{align}
and $p_{n} = 1$,
so
\begin{align}
e_n = -∑_{i=0}^{n-1} p_i e_i
\end{align}
so 
\begin{align}
X e_{n-1} = \frac{n}{ω} e_n = -\frac{n}{ω}∑_i p_i e_i
\end{align}


In [ ]:
def get_matrix(coeffs):
    ohm = 2*np.pi
    
    n = len(coeffs) + 1
    shift = np.arange(n-2, dtype=complex)+1
    matrix = np.diag(shift, k=-1)
    matrix[:,-1] = n*coeffs
    return matrix

In [ ]:
def get_expm1_matrix(n):
    coeffs = np.ones(n)
    coeffs[0] = 0
    return get_matrix(coeffs)/(2*np.pi)

In [ ]:
get_expm1_matrix(4)

In [ ]:
def plot_eigvals(es):
    plt.plot(np.real(es), np.imag(es), ls=' ', marker='o', ms=1)

In [ ]:
for size in range(50,350,50):
    plot_eigvals(np.linalg.eigvals(get_expm1_matrix(size)))
plt.grid(ls='-', alpha=.2)